In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import matplotlib as mpl
import os

from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC
import holodeck as holo

import hasasia.sim as hsim


In [ ]:
TARGET = 'hard_time' # EDIT AS NEEDED
TITLE = '$\\tau_\mathrm{hard}$'   # EDIT AS NEEDED
NVARS = 21

In [ ]:
path = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation/anatomy_09B'
load_data_from_file = path+f'/{TARGET}_{NVARS}vars_clbrt_reals.npz' 

if os.path.exists(load_data_from_file) is False:
    err = 'load data file does not exist, you need to construct it.'
    raise Exception(err)
print(load_data_from_file)
# print(save_data_to_file)

# Load Data from File

In [ ]:
file = np.load(load_data_from_file, allow_pickle=True)
print(file.files)
data = file['data']
params = file['params']
dsdat = file['dsdat']
file.close()

# Get Shapes, Edges, Frequency Info

In [ ]:
nfreqs, nreals, nloudest = [*data[0]['hc_ss'].shape]
fobs_cents = data[0]['fobs_cents']

In [ ]:
# mass edges
sam = holo.sams.Semi_Analytic_Model()
mm_edges = sam.mtot/MSOL

# frequency bin edges
ff_edges = data[0]['fobs_edges']/10**9

# get frequencies
ssfrq = np.repeat(fobs_cents, nreals*nloudest)/10**9 # nHz
bgfrq = np.repeat(fobs_cents, nreals)/10**9 # nHz

# Plot 1

In [ ]:
print(data[0]['sspar'].shape)

In [ ]:
dat = data[0]
parm = params[0]
for ii, dat in enumerate(data):
    parm = params[ii]
    # get masses
    ssmtt = dat['sspar'][0].flatten() /MSOL # msol
    bgmtt = dat['bgpar'][0].flatten() /MSOL # msol

    # get histograms
    hist_ss, mme, ffe = np.histogram2d(ssmtt, ssfrq, bins=(mm_edges, ff_edges))
    hist_bg, mme, ffe = np.histogram2d(bgmtt, bgfrq, bins=(mm_edges, ff_edges))

    fig, axs = plot.figax(ncols=2, ylabel='$M$ [M$_\odot$]', xlabel = '$f$ [nHz]', figsize=(8,4))
    ffgrid, mtgrid, = np.meshgrid(ffe, mme)
    # print(f"{mtgrid.shape=}, {ffgrid.shape=}")

    ax = axs[0]
    ax.set_title('$h_c^2$-weighted Background')
    ax.set_facecolor('k')
    im = ax.pcolormesh(ffgrid, mtgrid, np.log10(hist_bg), cmap='viridis')
    cbar = plt.colorbar(im, ax=ax, label='$\log N$', orientation='horizontal', pad=0.2)

    ax = axs[1]
    ax.set_title('Single Sources')
    ax.set_facecolor('k')
    ax.set_ylabel(None)
    im = ax.pcolormesh(ffgrid, mtgrid, np.log10(hist_ss), cmap='inferno')
    cbar = plt.colorbar(im, ax=ax, label='$\log N$', orientation='horizontal', pad=0.2)

    fig.suptitle((f"{TARGET}={parm[TARGET]:.2f}"))
    fig.tight_layout()


In [ ]:
print(parm)